We start by downloading the necessary libraries.







In [0]:
!git clone https://github.com/adines/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 398 (delta 0), reused 4 (delta 0), pack-reused 392
Receiving objects: 100% (398/398), 30.92 MiB | 33.65 MiB/s, done.
Resolving deltas: 100% (140/140), done.


Subsequently, we download the images from the ISBI challenge. 

In [0]:
!wget https://www.dropbox.com/s/30j6fugdwekgypd/unetData.zip?dl=0 -O unetData.zip
!unzip unetData.zip

--2019-01-28 15:57:45--  https://www.dropbox.com/s/30j6fugdwekgypd/unetData.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/30j6fugdwekgypd/unetData.zip [following]
--2019-01-28 15:57:45--  https://www.dropbox.com/s/raw/30j6fugdwekgypd/unetData.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3eda486d9109a8cfbd253a6a40.dl.dropboxusercontent.com/cd/0/inline/AaQNMA6Jibprr5m3VbgyjLZ8o5k65JPmJAPMmgsmCiZezT_AX3LofMe8Vn4qdZtbX0_BKZVDjhNlC8pY-OtnwsDlmxzIrmbBHL2YGpMqAdy54w/file# [following]
--2019-01-28 15:57:46--  https://uc3eda486d9109a8cfbd253a6a40.dl.dropboxusercontent.com/cd/0/inline/AaQNMA6Jibprr5m3VbgyjLZ8o5k65JPmJAPMmgsmCiZezT_AX3LofMe8Vn4qdZtbX0_BKZVDjhNlC8pY-OtnwsDlmxzIrmbBHL2YGpMqAdy54w/file
Resolving uc3e

We install some additional modules for computing some metrics. 

In [0]:
!pip install keras-metrics

We move to the unet folder.

In [0]:
cd unet/

/content/unet


We load the necessary modules.

In [0]:
from model import *
from data import *
import keras_metrics

Using TensorFlow backend.


We train the unet module with the original dataset.

In [0]:
data_gen_args = dict()
myGene = trainGenerator(2,'/content/unetData/train','images','labels',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=100,epochs=5,callbacks=[model_checkpoint])

/content/unet/model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Epoch 1/5
100/100 [==============================] - 49s 491ms/step - loss: 0.3761 - true_positive: 9917335.0000 - true_negative: 818475.0000 - false_positive: 2032115.0000 - false_negative: 339275.0000 - acc: 0.8191 - f1_score: 0.8932 - precision: 0.8299 - recall: 0.9669

Epoch 00001: loss improved from inf to 0.37606, saving model to unet_membrane.hdf5
Epoch 2/5
100/100 [==============================] - 42s 415ms/step - loss: 0.3116 - true_positive: 9564007.0000 - true_negative: 1845612.0000 - false_positive: 1004978.0000 - false_negative: 692603.0000 - acc: 0.8705 - f1_score: 0.9185 - precision: 0.9049 - recall: 0.9325

Epoch 00002: loss improved from 0.37606 to 0.31158, saving model to unet_membrane.hdf5
Epoch 3/5
100/100 [==============================] - 42s 416ms/step - loss: 0.2875 - true_positive: 9623118.0000 - true_negative: 2042973.0000 - false_positive: 807617.0000 - false_negative: 633492.000

Finally, we compute the different metrics. 

In [0]:
import cv2
import glob
import numpy as np
def geneTrainNpyPrueba(image_path,mask_path,flag_multi_class = False,num_class = 2,image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(image_path+"*")
    mask_name_arr = glob.glob(mask_path+"*")
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = cv2.imread(item,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(256,256))
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = cv2.imread(mask_name_arr[index],cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,(256,256))
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr

In [0]:
image_arr, mask_arr=geneTrainNpyPrueba('/content/unetData/test/images/','/content/unetData/test/labels/')
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.evaluate(image_arr,mask_arr)
print(results)

/content/unet/model.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


10/10 [==============================] - 5s 496ms/step
[0.2806262969970703, 492071, 103332, 31933, 28024, 0.9085128903388977, 0.9425753188113349, 0.9390596254988628, 0.9461175362191626]


In [0]:
TP=results[1]
TN=results[2]
FP=results[3]
FN=results[4]

In [0]:
TNR=TN/(TN+FP)
print(TNR)

0.7639226703138284


In [0]:
mkdir mkdir data/membrane/test2

In [0]:
def testGenerator(test_path,num_image = 10,target_size = (256,256),flag_multi_class = False,as_gray = True):
    image_name_arr = glob.glob(test_path+"*")
    for item in image_name_arr:
        img = io.imread(item,as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [0]:
testGene = testGenerator("/content/unetData/test/images/")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,10,verbose=1)
saveResult("data/membrane/test2",results)

/content/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


10/10 [==============================] - 1s 136ms/step


/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [0]:
ls data/membrane/test2

0_predict.png  2_predict.png  4_predict.png  6_predict.png  8_predict.png
1_predict.png  3_predict.png  5_predict.png  7_predict.png  9_predict.png


In [0]:
!zip -r imagesExp1.zip data/membrane/test2
from google.colab import files
files.download('imagesExp1.zip')

  adding: data/membrane/test2/ (stored 0%)
  adding: data/membrane/test2/9_predict.png (deflated 0%)
  adding: data/membrane/test2/3_predict.png (deflated 0%)
  adding: data/membrane/test2/8_predict.png (deflated 0%)
  adding: data/membrane/test2/6_predict.png (deflated 0%)
  adding: data/membrane/test2/1_predict.png (stored 0%)
  adding: data/membrane/test2/0_predict.png (deflated 0%)
  adding: data/membrane/test2/7_predict.png (deflated 0%)
  adding: data/membrane/test2/5_predict.png (deflated 0%)
  adding: data/membrane/test2/4_predict.png (deflated 0%)
  adding: data/membrane/test2/2_predict.png (deflated 0%)


In [0]:
!zip -r modelExp1.zip unet_membrane.hdf5
from google.colab import files
files.download('modelExp1.zip')

  adding: unet_membrane.hdf5 (deflated 7%)


----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 60504, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
!wget https://www.dropbox.com/s/r1sa7ldgqd0zftx/unet_membrane.hdf5?dl=0 -O unet_membrane.hdf5

--2019-01-23 11:53:57--  https://www.dropbox.com/s/r1sa7ldgqd0zftx/unet_membrane.hdf5?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r1sa7ldgqd0zftx/unet_membrane.hdf5 [following]
--2019-01-23 11:53:57--  https://www.dropbox.com/s/raw/r1sa7ldgqd0zftx/unet_membrane.hdf5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc24f7d38b1a4252af28fe226d7b.dl.dropboxusercontent.com/cd/0/inline/AZ_7ELaV6ZtRBbwcw3sF4cVofKpNNFML5BTJd_8TU_4bOC536A_GPgF1M8JmOEDiaALFnNag3mw74arONPjsT0yBa_WZ76nPjO9QEpNQbT-oEy4VOXmzV0P2PxxxixqyKl3gQUVf43MCFyAQGNTxukpybmmk7JfL1dNGTV8xXK9JLa0dabe811DcHcb1Oqnd3Eg/file# [following]
--2019-01-23 11:53:57--  https://uc24f7d38b1a4252af28fe226d7b.dl.dropboxusercontent.com/cd/0/inline/AZ_7ELaV6ZtRBbwcw3sF4cVofKp

In [0]:
ls

modelExp1.zip  sample_data/  unet/  unetData/  unetData.zip  unet_membrane.hdf5


In [0]:
!wget https://www.dropbox.com/s/n94i92sh80h69ja/imagesExp1.zip?dl=0 -O imagesExp1.zip
!unzip imagesExp1.zip

--2019-01-23 12:24:42--  https://www.dropbox.com/s/n94i92sh80h69ja/imagesExp1.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n94i92sh80h69ja/imagesExp1.zip [following]
--2019-01-23 12:24:42--  https://www.dropbox.com/s/raw/n94i92sh80h69ja/imagesExp1.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb45ea9c634e0411577b2ea7ee7.dl.dropboxusercontent.com/cd/0/inline/AZ_wbGBVg5XEru0EJQGwZ_vKmCqhVHrN8PhTJ-WYDH_GABDI5MMy_-tqdQSOY1IAbU_vreHwS-iuPm75sQ4xj9Ks4dkVJ85Kf3UFi8p-aJVRXUglhUm-B-B7KlKgppDghZL3stl1CvyJEBk_8HSO8z8KJn2Y7o15-nxFx-zPxcrJ3ll9eU_dpY9GUD_30uuZxpE/file# [following]
--2019-01-23 12:24:42--  https://ucb45ea9c634e0411577b2ea7ee7.dl.dropboxusercontent.com/cd/0/inline/AZ_wbGBVg5XEru0EJQGwZ_vKmCqhVHrN8PhTJ-W

In [0]:
ls /content/

data/           modelExp1.zip  unet/      unetData.zip
imagesExp1.zip  sample_data/   unetData/  unet_membrane.hdf5


In [0]:
import cv2
import glob
import numpy as np
def geneTrainNpyPrueba(image_path,mask_path,flag_multi_class = False,num_class = 2,image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(image_path+"*")
    mask_name_arr = glob.glob(mask_path+"*")
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = cv2.imread(item,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(256,256))
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = cv2.imread(mask_name_arr[index],cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,(256,256))
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr

In [0]:
image_arr, mask_arr=geneTrainNpyPrueba('/content/unetData/test/images/','/content/unetData/test/labels/')
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.evaluate(image_arr,mask_arr)
print(results)

/content/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


10/10 [==============================] - 5s 513ms/step
[1.0563255548477173, 0.9055343866348267]
